# Содержание
- [PCA](#1)
- [t-SNE](#2)

<a name='1'></a>
## PCA

PCA(Principal Component Analysis) - это метод снижения размерности данных, который используется для выявления наиболее информативных признаков в наборе данных и проекции их на новое пространство меньшей размерности. Основная цель PCA - найти линейные комбинации исходных признаков, называемые главными компонентами, которые содержат наибольшую дисперсию в данных.

Процесс работы PCA следующий:

1. Стандартизация данных: Исходные признаки стандартизируются, чтобы они имели среднее значение равное 0 и стандартное отклонение равное 1. Это делается для того, чтобы признаки с различными единицами измерения не искажали результаты анализа.
2. Вычисление матрицы ковариации: Вычисляется матрица ковариации, которая показывает связи исходных признаков друг с другом. Ковариация измеряет, насколько два признака меняются вместе.
3. Вычисление собственных значений и собственных векторов: Собственные значения и собственные векторы извлекаются из матрицы ковариации. Собственные значения представляют собой меру дисперсии вдоль соответствующих собственных векторов.
4. Сортировка главных компонент: Главные компоненты сортируются по убыванию их собственных значений. Главная компонента с наибольшим собственным значением содержит наибольшую дисперсию в данных.
5. Выбор количества главных компонент: Определяется количество главных компонент, которые будут использоваться для проекции данных на новое пространство. Можно выбрать определенное количество компонент или определить процент дисперсии, которую они объясняют.
6. Проекция данных: Исходные признаки проецируются на выбранные главные компоненты, формируя новое пространство с меньшей размерностью.

Результат PCA - это новое пространство признаков с меньшей размерностью, где каждая главная компонента представляет собой линейную комбинацию исходных признаков. Главные компоненты упорядочены по убыванию их значимости, и первые компоненты содержат наибольшую долю дисперсии в данных. PCA широко используется для визуализ

ации данных, устранения мультиколлинеарности, сжатия данных и улучшения производительности алгоритмов машинного обучения путем снижения размерности пространства признаков.

In [ ]:
import numpy as np

class PCA:
    def __init__(self, n_components):
        self.n_components = n_components
        self.components = None
        self.mean = None

    def fit(self, X):
        # Вычисляем среднее значение каждого признака
        self.mean = np.mean(X, axis=0)

        # Центрируем данные путем вычитания среднего
        X_centered = X - self.mean

        # Вычисляем матрицу ковариации
        covariance_matrix = np.cov(X_centered.T)

        # Вычисляем собственные значения и собственные векторы
        eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)

        # Сортируем собственные значения и соответствующие собственные векторы в порядке убывания
        eigen_indices = np.argsort(eigenvalues)[::-1]
        sorted_eigenvectors = eigenvectors[:, eigen_indices]

        # Выбираем первые n_components собственных векторов
        self.components = sorted_eigenvectors[:, :self.n_components]

    def transform(self, X):
        # Центрируем данные путем вычитания среднего
        X_centered = X - self.mean

        # Проецируем данные на главные компоненты
        transformed = np.dot(X_centered, self.components)

        return transformed

In [ ]:
# Пример использования
X = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]])
n_components = 2

pca = PCA(n_components)
pca.fit(X)

transformed = pca.transform(X)
print("Преобразованные данные:")
print(transformed)

Рассмотрим пример на реальных данных и визуализацию, чтобы увидеть, как библиотечный PCA работает в реальной задаче. Используем набор данных Iris, который часто применяется для демонстрации анализа данных. PCA поможет нам визуализировать многомерные данные в двухмерном пространстве.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import load_iris

# Загружаем данные Iris
data = load_iris()
X_iris = data.data  # признаки
y_iris = data.target  # метки классов
target_names = data.target_names

# Стандартизация данных
scaler = StandardScaler()
X_iris_scaled = scaler.fit_transform(X_iris)

# Применяем PCA для снижения размерности до 2D
pca_iris = PCA(n_components=2)
X_iris_pca = pca_iris.fit_transform(X_iris_scaled)

# Визуализируем результаты
plt.figure(figsize=(8, 6))
colors = ['coral', 'darkkhaki', 'teal']

for color, i, target_name in zip(colors, [0, 1, 2], target_names):
    plt.scatter(X_iris_pca[y_iris == i, 0], X_iris_pca[y_iris == i, 1], color=color, alpha=0.8, label=target_name)

plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title("PCA на данных Iris")
plt.xlabel("Первая главная компонента")
plt.ylabel("Вторая главная компонента")
plt.show()


Вопросы для обсуждения:

- Что вы замечаете в распределении классов?
- Есть ли классы, которые пересекаются?
- Какое преимущество даёт снижение размерности для визуализации и анализа?
- Могут ли потери информации при снижении размерности повлиять на результаты анализа?

Посмотрим, как меняется точность обученной модели при использовании данных пониженной размерности. Ожидание: поскольку данные хорошо разделяются визуально, скорее всего потери в качестве не будет.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_iris_scaled, y_iris, test_size=0.3, random_state=42)

# Модель без PCA
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_no_pca = accuracy_score(y_test, y_pred)

# Модель с PCA
X_train_pca = pca_iris.transform(X_train)
X_test_pca = pca_iris.transform(X_test)
clf.fit(X_train_pca, y_train)
y_pred_pca = clf.predict(X_test_pca)
accuracy_with_pca = accuracy_score(y_test, y_pred_pca)

print(f"Точность без PCA: {accuracy_no_pca:.2f}")
print(f"Точность с PCA: {accuracy_with_pca:.2f}")

Рассмотрим более сложные данные: набор изображений с цифрами от 0 до 9.

In [ ]:
from sklearn.datasets import load_digits

# Загружаем данные
digits = load_digits()
X_digits = digits.data
y_digits = digits.target

# Применяем PCA
pca_digits = PCA(n_components=2)
X_digits_pca = pca_digits.fit_transform(X_digits)

# Визуализация
plt.figure(figsize=(10, 8))
for i in range(10):
    plt.scatter(X_digits_pca[y_digits == i, 0], X_digits_pca[y_digits == i, 1], label=f'Цифра {i}', alpha=0.6)
plt.legend()
plt.title("PCA для набора данных Digits")
plt.xlabel("Первая главная компонента")
plt.ylabel("Вторая главная компонента")
plt.show()


Разделение между классами гораздо менее очевидное. Можно ожидать снижение точности при обучении модели на данных пониженной размерности.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_digits, y_digits, test_size=0.4, random_state=42)

# Модель без PCA
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_no_pca = accuracy_score(y_test, y_pred)

# Модель с PCA
X_train_pca = pca_digits.transform(X_train)
X_test_pca = pca_digits.transform(X_test)
clf.fit(X_train_pca, y_train)
y_pred_pca = clf.predict(X_test_pca)
accuracy_with_pca = accuracy_score(y_test, y_pred_pca)

print(f"Точность без PCA: {accuracy_no_pca:.2f}")
print(f"Точность с PCA: {accuracy_with_pca:.2f}")

Падение точности после применения PCA — вполне ожидаемое явление, особенно если происходит значительная потеря информации при снижении размерности.
Но это снижение в точности происходит за счёт выигрыша во времени обучения. Посмотрим, как меняются точность и время затраченное на обучение в зависимости от числа компонент.

In [ ]:
import time

In [ ]:
import matplotlib.pyplot as plt

# Данные для примера (используем результаты из цикла с компонентами)
# components_range = range(1, 51, 5)  # Число компонент PCA
components_range = range(1, X_train.shape[1] + 1)
training_times = []  # Список для времени обучения
accuracies = []  # Список для точности модели

for n in components_range:
    pca = PCA(n_components=n)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)

    start_time = time.time()
    clf = RandomForestClassifier(random_state=42)
    clf.fit(X_train_pca, y_train)
    y_pred = clf.predict(X_test_pca)
    training_times.append(time.time() - start_time)
    accuracies.append(accuracy_score(y_test, y_pred))

# Построение графика
fig, ax1 = plt.subplots(figsize=(10, 6))

# Левая ось: точность
ax1.set_xlabel('Число компонент PCA')
ax1.set_ylabel('Точность', color='blue')
ax1.plot(components_range, accuracies, marker='o', color='blue', label='Точность')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.grid(visible=True, linestyle='--', alpha=0.5)

# Правая ось: время обучения
ax2 = ax1.twinx()
ax2.set_ylabel('Время обучения (с)', color='orange')
ax2.plot(components_range, training_times, marker='s', color='orange', label='Время обучения')
ax2.tick_params(axis='y', labelcolor='orange')

# Заголовок и легенда
fig.suptitle('Влияние числа компонент PCA на точность и время обучения', fontsize=14)
fig.tight_layout()

# Отображение графика
plt.show()


В данном случае, точность в обучении между полноразмерными данными и данными с размерностью, уменьшенной до 15-20 компонент практически не меняется, при этом время обучения на данных меньшей размерности оказыватеся в полтора-два раза короче.

<a name='2'></a>
## t-SNE

t-SNE (t-Distributed Stochastic Neighbor Embedding) - это метод снижения размерности данных, который используется для визуализации сложных структур данных и обнаружения скрытых паттернов. В отличие от PCA, t-SNE обеспечивает сохранение не только линейной структуры данных, но и нелинейных отношений между точками.

Процесс работы t-SNE следующий:

1. Вычисление аффинности: Сначала вычисляется аффинность (похожесть) между парами точек данных. Это может быть сделано с использованием Гауссовой функции, основанной на расстоянии между точками в исходном пространстве.
2. Вычисление условной вероятности: Для каждой точки данных вычисляется условная вероятность, которая показывает вероятность выбрать другую точку в качестве соседа, исходя из аффинности. Более похожие точки имеют более высокие вероятности быть выбранными в качестве соседей.
3. Определение сходства в пространстве низкой размерности: Для пространства низкой размерности (обычно 2D) исходные точки данных и их аффинности переопределяются с использованием условных вероятностей. Это позволяет сохранить близость точек, имеющих высокую аффинность в исходном пространстве.
4. Минимизация дивергенции Кульбака-Лейблера: t-SNE оптимизирует распределение точек в пространстве низкой размерности, минимизируя дивергенцию Кульбака-Лейблера между условными вероятностями точек в исходном пространстве и пространстве низкой размерности.
5. Итерационная оптимизация: Алгоритм итеративно обновляет расположение точек в пространстве низкой размерности, чтобы минимизировать дивергенцию Кульбака-Лейблера. Оптимизация основывается на градиентных методах или методах случайного блуждания.

Результат t-SNE представляет собой вложение точек в пространство низкой размерности, где близкие точки соответствуют точкам с высокой аффинностью в исходном пространстве. t-SNE обладает способностью выявлять сложные нелинейные структуры и кластеры в данных, что делает его полезным для визуализации и понимания сложных наборов данных. Однако важно отметить, что расположение точек в пространстве низкой размерности может быть чувствительным к различным параметрам и инициализации, поэтому результаты t-SNE требуют внимательного анализа и интерпретации.

In [ ]:
import numpy as np
from sklearn.decomposition import PCA

class TSNE:
    def __init__(self, n_components=2, perplexity=30, learning_rate=200, n_iter=1000, random_state=None):
        """
        Инициализация параметров t-SNE.
        n_components: размерность целевого пространства (обычно 2 или 3).
        perplexity: целевая сложность (число "эффективных" соседей для каждого объекта).
        learning_rate: скорость обучения (шаг градиентного спуска).
        n_iter: количество итераций оптимизации.
        random_state: начальное значение для генератора случайных чисел.
        """
        self.n_components = n_components
        self.perplexity = perplexity
        self.learning_rate = learning_rate
        self.n_iter = n_iter
        self.random_state = random_state

    def _compute_pairwise_affinities(self, X):
        """
        Расчет матрицы вероятностей P в высокоразмерном пространстве на основе perplexity.
        Используется бинарный поиск для подбора параметра sigma (β = 1 / sigma^2).
        """
        n = X.shape[0]  # Количество точек
        distances = np.square(X[:, np.newaxis] - X).sum(axis=2)  # Квадрат Евклидовых расстояний
        sigmas = np.ones(n)  # Инициализация сигм
        P = np.zeros((n, n))  # Матрица вероятностей P

        for i in range(n):
            # Бинарный поиск для настройки sigma
            beta_min, beta_max = None, None
            beta = 1.0
            for _ in range(50):  # Максимум 50 итераций поиска
                P_i = np.exp(-distances[i] * beta)  # Вычисление вероятностей
                P_i[i] = 0  # Зануляем вероятность для самого себя
                sum_P_i = P_i.sum()  # Сумма вероятностей
                H_i = np.log(sum_P_i) + beta * (distances[i] * P_i).sum() / sum_P_i  # Энтропия
                P[i] = P_i / sum_P_i  # Нормализация

                # Проверка разницы между текущей и целевой энтропией
                H_diff = H_i - np.log(self.perplexity)
                if np.abs(H_diff) < 1e-5:  # Если достигнута точность
                    break
                if H_diff > 0:  # Если текущая энтропия больше целевой
                    beta_min = beta
                    beta = (beta + beta_max) / 2 if beta_max else beta * 2
                else:  # Если текущая энтропия меньше целевой
                    beta_max = beta
                    beta = (beta + beta_min) / 2 if beta_min else beta / 2

        # Симметризация P и нормализация
        P = (P + P.T) / (2 * n)
        return P

    def _compute_joint_probabilities_q(self, Y):
        """
        Расчет вероятностей Q в низкоразмерном пространстве.
        """
        distances = np.square(Y[:, np.newaxis] - Y).sum(axis=2)  # Квадрат расстояний в низкоразмерном пространстве
        Q = 1 / (1 + distances)  # Вычисление Q по формуле t-распределения
        np.fill_diagonal(Q, 0)  # Обнуляем диагональные элементы
        Q /= Q.sum()  # Нормализация Q
        return Q

    def _compute_gradient(self, P, Q, Y):
        """
        Вычисление градиента Kullback-Leibler дивергенции.
        """
        pq_diff = P - Q  # Разница между высоко- и низкоразмерными вероятностями
        distances = 1 + np.square(Y[:, np.newaxis] - Y).sum(axis=2)  # 1 + ||y_i - y_j||^2
        weights = pq_diff[:, :, np.newaxis] / distances[:, :, np.newaxis]  # Вес градиента
        grad = 4 * (weights * (Y[:, np.newaxis] - Y)).sum(axis=1)  # Итоговый градиент
        return grad

    def fit_transform(self, X):
        """
        Основной метод: выполнение алгоритма t-SNE.
        Возвращает преобразованные точки в низкоразмерном пространстве.
        """
        np.random.seed(self.random_state)  # Фиксация генератора случайных чисел

        # Шаг 1: расчет матрицы P
        P = self._compute_pairwise_affinities(X)
        P *= 4  # Раннее увеличение (early exaggeration)

        # Шаг 2: Инициализация точек в низкоразмерном пространстве
        Y = np.random.normal(0, 1e-4, (X.shape[0], self.n_components))

        # Итеративная оптимизация
        for iter in range(self.n_iter):
            Q = self._compute_joint_probabilities_q(Y)  # Расчет Q
            grad = self._compute_gradient(P, Q, Y)  # Расчет градиента

            # Градиентный спуск
            Y -= self.learning_rate * grad

            # Уменьшение раннего увеличения после 250 итераций
            if iter == 250:
                P /= 4

            # Логирование на каждой сотой итерации
            if iter % 100 == 0 or iter == self.n_iter - 1:
                kl_div = np.sum(P * np.log((P + 1e-12) / (Q + 1e-12)))  # Расчет KL-дивергенции
                print(f"Итерация {iter}, KL-дивергенция: {kl_div:.5f}")

        return Y


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris, load_digits
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE as SKLEARN_TSNE  # Для сравнения с вашим методом


# Функция для визуализации
def plot_tsne(embedding, labels, title):
    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(embedding[:, 0], embedding[:, 1], c=labels, cmap='viridis', alpha=0.7)
    plt.colorbar(scatter, label="Классы")
    plt.title(title, fontsize=16)
    plt.xlabel("Первая компонента", fontsize=12)
    plt.ylabel("Вторая компонента", fontsize=12)
    plt.show()

# Данные Iris
iris = load_iris()
X_iris = iris.data
y_iris = iris.target

# Данные Digits
digits = load_digits()
X_digits = digits.data
y_digits = digits.target

# Стандартизация данных
scaler = StandardScaler()
X_iris_scaled = scaler.fit_transform(X_iris)
X_digits_scaled = scaler.fit_transform(X_digits)

# Применение кастомного TSNE для Iris
tsne_custom_iris = TSNE(n_components=2, perplexity=30, learning_rate=200, n_iter=500)
X_embedded_iris = tsne_custom_iris.fit_transform(X_iris_scaled)
plot_tsne(X_embedded_iris, y_iris, "t-SNE на данных Iris (кастомный)")

# Сравнение с sklearn t-SNE
sklearn_tsne_iris = SKLEARN_TSNE(n_components=2, perplexity=30, learning_rate=200, max_iter=500, random_state=0)
embedding_sklearn_iris = sklearn_tsne_iris.fit_transform(X_iris_scaled)
plot_tsne(embedding_sklearn_iris, y_iris, "t-SNE на данных Iris (sklearn)")

# Применение кастомного TSNE для Digits
tsne_custom_digits = TSNE(n_components=2, perplexity=5, learning_rate=200, n_iter=250)
X_embedded_digits = tsne_custom_digits.fit_transform(X_digits_scaled)
plot_tsne(X_embedded_digits, y_digits, "t-SNE на данных Digits (кастомный)")

# Сравнение с sklearn t-SNE
sklearn_tsne_digits = SKLEARN_TSNE(n_components=2, perplexity=5, learning_rate=200, max_iter=250, random_state=0)
embedding_sklearn_digits = sklearn_tsne_digits.fit_transform(X_digits_scaled)
plot_tsne(embedding_sklearn_digits, y_digits, "t-SNE на данных Iris (sklearn)")

Теперь сравним эффективность работы библиотечных имплементаций t-SNE и PCA.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris, load_digits
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Настройки для визуализаций
sns.set(style="whitegrid", font_scale=1.2, palette="Set2")

# Применение t-SNE
tsne_iris = TSNE(n_components=2, perplexity=30, random_state=42)
X_iris_tsne = tsne_iris.fit_transform(X_iris_scaled)

# Сравнение PCA и t-SNE
pca_iris = PCA(n_components=2)
X_iris_pca = pca_iris.fit_transform(X_iris_scaled)

fig, ax = plt.subplots(1, 2, figsize=(12, 5))

# PCA
sns.scatterplot(
    x=X_iris_pca[:, 0], y=X_iris_pca[:, 1], hue=y_iris, palette="viridis", ax=ax[0], s=50, alpha=0.7
)
ax[0].set_title("PCA Visualization of Iris Dataset", fontsize=14)
ax[0].set_xlabel("Principal Component 1")
ax[0].set_ylabel("Principal Component 2")
# ax[0].legend(title="Classes", labels=iris.target_names)

# t-SNE
sns.scatterplot(
    x=X_iris_tsne[:, 0], y=X_iris_tsne[:, 1], hue=y_iris, palette="viridis", ax=ax[1], s=50, alpha=0.7
)
ax[1].set_title("t-SNE Visualization of Iris Dataset", fontsize=14)
ax[1].set_xlabel("t-SNE Component 1")
ax[1].set_ylabel("t-SNE Component 2")
# ax[1].legend(title="Classes", labels=iris.target_names)

plt.tight_layout()
plt.show()

# Работа с digits dataset

# Применение t-SNE
tsne_digits = TSNE(n_components=2, perplexity=30, random_state=42)
X_digits_tsne = tsne_digits.fit_transform(X_digits_scaled)

# Сравнение PCA и t-SNE
pca_digits = PCA(n_components=2)
X_digits_pca = pca_digits.fit_transform(X_digits_scaled)

fig, ax = plt.subplots(1, 2, figsize=(12, 5))

# Визуализация t-SNE результатов
sns.scatterplot(
    x=X_digits_pca[:, 0], y=X_digits_pca[:, 1], hue=y_digits, palette="tab10", ax=ax[0], s=50, alpha=0.7
)
ax[0].set_title("PCA Visualization of Digits Dataset", fontsize=16)
ax[0].set_xlabel("PCA Component 1")
ax[0].set_ylabel("PCA Component 2")
# plt.legend(title="Digits", loc="center left", bbox_to_anchor=(1, 0.5))


# Визуализация t-SNE результатов
sns.scatterplot(
    x=X_digits_tsne[:, 0], y=X_digits_tsne[:, 1], hue=y_digits, palette="tab10", ax=ax[1], s=50, alpha=0.7
)
ax[1].set_title("t-SNE Visualization of Digits Dataset", fontsize=16)
ax[1].set_xlabel("t-SNE Component 1")
ax[1].set_ylabel("t-SNE Component 2")
# plt.legend(title="Digits", loc="center left", bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris, load_digits
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import time

# Настройки для визуализаций
sns.set(style="whitegrid", font_scale=1.2, palette="Set2")

# --- Работа с Iris dataset ---
# Измерение времени для t-SNE
start_tsne = time.time()
tsne_iris = TSNE(n_components=2, perplexity=30, random_state=42)
X_iris_tsne = tsne_iris.fit_transform(X_iris_scaled)
time_tsne_iris = time.time() - start_tsne

# Измерение времени для PCA
start_pca = time.time()
pca_iris = PCA(n_components=2)
X_iris_pca = pca_iris.fit_transform(X_iris_scaled)
time_pca_iris = time.time() - start_pca

# Визуализация результатов для Iris dataset
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

# PCA
sns.scatterplot(
    x=X_iris_pca[:, 0], y=X_iris_pca[:, 1], hue=y_iris, palette="viridis", ax=ax[0], s=50, alpha=0.7
)
ax[0].set_title(f"PCA Visualization of Iris Dataset\n(Time: {time_pca_iris:.2f} seconds)", fontsize=14)
ax[0].set_xlabel("Principal Component 1")
ax[0].set_ylabel("Principal Component 2")

# t-SNE
sns.scatterplot(
    x=X_iris_tsne[:, 0], y=X_iris_tsne[:, 1], hue=y_iris, palette="viridis", ax=ax[1], s=50, alpha=0.7
)
ax[1].set_title(f"t-SNE Visualization of Iris Dataset\n(Time: {time_tsne_iris:.2f} seconds)", fontsize=14)
ax[1].set_xlabel("t-SNE Component 1")
ax[1].set_ylabel("t-SNE Component 2")

plt.tight_layout()
plt.show()

# --- Работа с Digits dataset ---
# Измерение времени для t-SNE
start_tsne = time.time()
tsne_digits = TSNE(n_components=2, perplexity=30, random_state=42)
X_digits_tsne = tsne_digits.fit_transform(X_digits_scaled)
time_tsne_digits = time.time() - start_tsne

# Измерение времени для PCA
start_pca = time.time()
pca_digits = PCA(n_components=2)
X_digits_pca = pca_digits.fit_transform(X_digits_scaled)
time_pca_digits = time.time() - start_pca

# Визуализация результатов для Digits dataset
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

# PCA
sns.scatterplot(
    x=X_digits_pca[:, 0], y=X_digits_pca[:, 1], hue=y_digits, palette="tab10", ax=ax[0], s=50, alpha=0.7
)
ax[0].set_title(f"PCA Visualization of Digits Dataset\n(Time: {time_pca_digits:.2f} seconds)", fontsize=16)
ax[0].set_xlabel("PCA Component 1")
ax[0].set_ylabel("PCA Component 2")

# t-SNE
sns.scatterplot(
    x=X_digits_tsne[:, 0], y=X_digits_tsne[:, 1], hue=y_digits, palette="tab10", ax=ax[1], s=50, alpha=0.7
)
ax[1].set_title(f"t-SNE Visualization of Digits Dataset\n(Time: {time_tsne_digits:.2f} seconds)", fontsize=16)
ax[1].set_xlabel("t-SNE Component 1")
ax[1].set_ylabel("t-SNE Component 2")

plt.tight_layout()
plt.show()


In [ ]:
Из этих примеров видно, что t-SNE более продуктивно производит снижение размерности данных, но требует больше времени на реализацию:

In [ ]:
# Печать времени выполнения
print("Время выполнения для набора данных Iris:")
print(f"PCA: {time_pca_iris:.4f} секунд")
print(f"t-SNE: {time_tsne_iris:.4f} секунд")

print("\nВремя выполнения для набора данных Digits:")
print(f"PCA: {time_pca_digits:.4f} секунд")
print(f"t-SNE: {time_tsne_digits:.4f} секунд")

t-SNE можно использовать для визуализации многомерных данных, но для обучения моделей он не подходит: мы не можем применить ранее обученную модель t-SNE к новым данным, поскольку для вычисления t-SNE требуется весь набор данных для оценки нелинейных зависимостей.
